In [1]:
#!pip install ipywidgets
#!jupyter nbextension enable --py widgetsnbextension
import scipy as sp
import scipy.io
import scipy.stats
import os
import numpy as np
import pandas as pd
import glob
import csv
import random as rand
from tqdm import tnrange, tqdm_notebook

import cmocean
import matplotlib.patches as patches
from matplotlib import gridspec

from collections import Iterable
import matplotlib.pylab as mpl

import random as rand
from ipywidgets import *
import colorlover as cl

from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize


C:\Users\Eric\AppData\Local\Temp/ipykernel_8484/2638047130.py:18: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable


In [2]:
font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 22}

mpl.rc('font', **font)
mpl.rc('xtick', labelsize=20) 
mpl.rc('ytick', labelsize=20)
mpl.rc('axes', labelsize=20)


bin_size = 0.025
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

## Load behavioral data

In [3]:
os.getcwd()

'C:\\Users\\Eric\\Documents\\09-12-2021\\crossmodal\\Figure1'

In [4]:
beh_data_files = glob.glob("behavior*")
behavior_df = pd.DataFrame([])
mat = sp.io.loadmat(beh_data_files[1])
beh_df = pd.DataFrame(mat['behavior_log'])

In [5]:
for col in range(beh_df.shape[1]): 
    beh_df.iloc[:,col] = beh_df.iloc[:,col].str[0]
    
col_names = beh_df.loc[0,:]
beh_df = beh_df.iloc[1:,:].rename(columns = col_names)

for col in [5,8]:
    beh_df.iloc[:,col] = beh_df.iloc[:,col].str[0]

beh_df.head()

,mouse_name,session_type,session_date,block_type,trial_type,trial_num,som_stim_type,vis_stim_type,response
1,EF0072,Solo.EFcross2Array,12-02-15,Visual,NoStim_Som_NoCue,1,SineAmp1Freq40Cyc40,Amp0p05Dur0p075,0
2,EF0072,Solo.EFcross2Array,12-02-15,Whisker,Stim_Som_NoCue,2,SineAmp1Freq40Cyc40,Amp0p05Dur0p075,0
3,EF0072,Solo.EFcross2Array,12-02-15,Whisker,NoStim_Som_NoCue,3,SineAmp1Freq40Cyc40,Amp0p05Dur0p075,1
4,EF0072,Solo.EFcross2Array,12-02-15,Whisker,Stim_Som_NoCue,4,SineAmp1Freq40Cyc40,Amp0p05Dur0p075,0
5,EF0072,Solo.EFcross2Array,12-02-15,Whisker,NoStim_Som_NoCue,5,SineAmp1Freq40Cyc40,Amp0p05Dur0p075,1


In [6]:
beh_df['mouse_name'].unique()

array(['EF0072', 'EF0074', 'EF0076', 'EF0077', 'EF0079', 'Ef0079',
       'EF0081', 'EF0083', 'EF0084', 'EF0085', 'EF0088', 'EF0089',
       'EF0091', 'EF0094', 'EF0098', 'EF0099', 'EF0100', 'EF0101',
       'EF00101', 'EF0102', 'EF0111', 'EF0112', 'EF0114', 'EF0131',
       'EF0132', 'EF0134', 'EF0144', 'EF0147', 'EF0149', 'EF0148',
       'EF0151'], dtype=object)

#### clean up dataframe - remove unsuccessful mice, rename misnamed mice

In [7]:
beh_df = beh_df.loc[beh_df['mouse_name'] != 'EF0072', :]
beh_df = beh_df.loc[beh_df['mouse_name'] != 'EF0100', :]
beh_df.loc[beh_df['mouse_name'] == 'EF00101', 'mouse_name'] = 'EF0101'
beh_df.loc[beh_df['mouse_name'] == 'Ef0079', 'mouse_name'] = 'EF0079'

#### Note: EF0094,EF0098 were switch-task mice from the start and therefore will be excluded from the behavioral analyses

In [9]:
beh_df = beh_df.loc[beh_df['mouse_name'] != 'EF0094', :]
beh_df = beh_df.loc[beh_df['mouse_name'] != 'EF0098', :]
beh_df = beh_df.loc[beh_df['mouse_name'] != 'EF0151', :] # not included in any experiments
beh_df = beh_df.loc[beh_df['mouse_name'] != 'EF0134', :] # not included in any experiments

In [10]:
beh_df['mouse_name'].unique()

array(['EF0074', 'EF0076', 'EF0077', 'EF0079', 'EF0081', 'EF0083',
       'EF0084', 'EF0085', 'EF0088', 'EF0089', 'EF0091', 'EF0099',
       'EF0101', 'EF0102', 'EF0111', 'EF0112', 'EF0114', 'EF0131',
       'EF0132', 'EF0144', 'EF0147', 'EF0149', 'EF0148'], dtype=object)

#### remove premature lick trials

In [11]:
beh_df = beh_df.loc[beh_df['response'].isin([0,1,2]).as_matrix(), :]
beh_df = beh_df.sort_values(by = ['mouse_name', 'session_date', 'trial_num'])

AttributeError: 'Series' object has no attribute 'as_matrix'

#### remove trials following block transitions so that it is not taken into account in performance measurement - these are trials where the mouse is figuring out through trial and error that the block has switched

In [11]:
beh_df = beh_df.sort_values(by = ['mouse_name', 'session_date', 'trial_num']).reset_index()
unique_sessions = beh_df[['mouse_name', 'session_date']].drop_duplicates().reset_index(drop =True)

all_block_trans=[]
for session_row in range(unique_sessions.shape[0]):
    session = unique_sessions.loc[session_row].to_frame().T
    session_subset = pd.merge(beh_df, session, on = ['mouse_name', 'session_date'], 
                              how = 'inner').set_index('index')
    block_trans = session_subset.index[np.where((np.diff((session_subset['block_type'] == 'Visual')*1)*-1))[0]+1]
    block_trans = np.array([(x, x+1, x+2, x+3, x+4) for x in block_trans]).flatten()
    all_block_trans.append(block_trans)
    
beh_df = beh_df.loc[~beh_df['index'].isin(np.concatenate(all_block_trans)),:]
# session_subset.loc[~session_subset['trial_num'].isin(block_trans), 'correct'].mean()

In [12]:
from datetime import datetime
beh_df['session_date'] = beh_df['session_date'].apply(lambda x:  datetime.strptime(x, '%m-%d-%y'))
beh_df = beh_df.sort_values(by = 'session_date').reset_index(drop = True)

#### define which days are the first full crossmodal training day for each mouse

In [13]:
first_CM_days = pd.DataFrame({'EF0074':'12-10-15', 'EF0076': '12-23-15', 'EF0077': '02-11-16', 'EF0079' : '02-12-16',  
                              'EF0081': '04-12-16','EF0083': '05-10-16', 'EF0084': '06-13-16', 'EF0085': '06-13-16',
                              'EF0088':'06-28-16',  'EF0089':'07-08-16','EF0091': '12-01-16', 'EF0094': '01-24-17',
                              'EF0098': '02-07-17' , 'EF0099': '02-08-17', 'EF0101' : '06-20-17', 'EF0102' : '06-20-17',
                             'EF0111':'02-20-18', 'EF0112':'02-20-18', 'EF0114':'03-12-18', 'EF0131': '06-14-18', 'EF0132':'06-21-18',
                             'EF0144':'08-01-18', 'EF0147':'10-22-18', 'EF0148':'10-22-18', 'EF0149':'10-23-18', 'EF0151':'11-02-18'}, 
                             index = ['first_train_date']).T

first_CM_days['first_train_date'] = first_CM_days['first_train_date'].apply(lambda x:  datetime.strptime(x, '%m-%d-%y'))


#### load main data table that includes ephys data. This will define which days are the test days/recorded sessions for each mouse

In [14]:
pd.read_hdf('log_df_rev.h5', 'table')

FileNotFoundError: File log_df_rev.h5 does not exist

In [ ]:
log_df = pd.read_hdf('log_df_rev.h5', 'table')
switch_log_df = pd.read_hdf('switching_log.h5', 'table')

In [ ]:
log_df['stim_onset'] = log_df['stim_onset'].fillna(0)
log_df['spike_times(stim_aligned)'] = log_df['spike_times'] - log_df['stim_onset']
log_df = log_df[~log_df['trial_type'].str.contains('NoStim')]
licks = pd.concat([log_df['licks_right'] - log_df['stim_onset'] , log_df['licks_left']-log_df['stim_onset']], axis=1)
licks = licks.applymap(lambda y: y[[0.1<y]] if len(y) > 0 else y)
licks = licks.applymap(lambda y: y[[3>=y]] if len(y) > 0 else y)
first_licks = licks.applymap(lambda y: min(y) if len(y) > 0 else np.nan)
last_licks = licks.applymap(lambda y: max(y) if len(y) > 0 else np.nan)

log_df['first_lick'] = first_licks.min(axis=1)
log_df['correct'] = 0

# fwd contingencies - correct responses
log_df.loc[~(log_df['mouse_name'].isin(['EF0091', 'EF0099', 'EF0101', 'EF0102'])) & (log_df['block_type'] == 'Whisker') &
       (log_df['trial_type'].str.contains('Som')) & (log_df['response'] == 1), 'correct'] = 1
log_df.loc[~(log_df['mouse_name'].isin(['EF0091', 'EF0099', 'EF0101', 'EF0102'])) & (log_df['block_type'] == 'Visual') &
       (log_df['trial_type'].str.contains('Vis')) & (log_df['response'] == 2), 'correct'] = 1

#rev contingencies - correct responses
log_df.loc[(log_df['mouse_name'].isin(['EF0091', 'EF0099', 'EF0101', 'EF0102'])) & (log_df['block_type'] == 'Whisker') &
       (log_df['trial_type'].str.contains('Som')) & (log_df['response'] == 2), 'correct'] = 1
log_df.loc[(log_df['mouse_name'].isin(['EF0091', 'EF0099', 'EF0101', 'EF0102'])) & (log_df['block_type'] == 'Visual') &
       (log_df['trial_type'].str.contains('Vis')) & (log_df['response'] == 1), 'correct'] = 1

#correct rejections
log_df.loc[(log_df['block_type'] == 'Whisker') & (log_df['trial_type'].str.contains('Vis'))
           & (log_df['response'] == 0), 'correct'] = 1
log_df.loc[(log_df['block_type'] == 'Visual') & (log_df['trial_type'].str.contains('Som'))
           & (log_df['response'] == 0), 'correct'] = 1

In [ ]:
# excluded EF0088 since trained on switching task after initial CM training
switch_log_df = switch_log_df.loc[~(switch_log_df['mouse_name'] == 'EF0088'),:]

In [ ]:
recording_days_sw = switch_log_df.loc[:, ['mouse_name', 'date']].drop_duplicates()
recording_days = pd.concat((log_df.loc[:, ['mouse_name', 'date']].drop_duplicates(), recording_days_sw), axis = 0)


In [ ]:
recording_days['date'] = recording_days['date'].apply(lambda x:  datetime.strptime(x, '%m-%d-%y'))

first_recording = {}
for mouse in recording_days['mouse_name'].unique():
    first = recording_days.loc[recording_days['mouse_name'] == mouse, 'date'].min()
    first_recording[mouse] = first
first_recording = pd.DataFrame(first_recording, index = ['first_rec_date']).T


In [ ]:
beh_df['full_stim'] = 0
beh_df.loc[~beh_df['trial_type'].str.contains('1Cyc'), 'full_stim'] = 1

In [ ]:
short_stim_performance = pd.pivot_table(beh_df[beh_df['full_stim'] == 0], index = ['mouse_name', 'session_date'],
                                       values = 'correct', aggfunc = np.mean)
full_stim_performance = pd.pivot_table(beh_df[beh_df['full_stim'] == 1], index = ['mouse_name', 'session_date'],
                                       values = 'correct', aggfunc = np.mean)
full_stim_performance.head()

In [ ]:
block_performance_short = pd.pivot_table(beh_df[beh_df['full_stim'] == 0], index = ['mouse_name', 'session_date'],
                                         columns = ['block_type'], values = 'correct', aggfunc = np.mean)
block_performance_full = pd.pivot_table(beh_df[beh_df['full_stim'] == 1], index = ['mouse_name', 'session_date'],
                                        columns = ['block_type'], values = 'correct', aggfunc = np.mean)
block_performance_full.head()

#### calculate the proportion of trials that are hits, misses, FA, and CR for each mouse during the recording sessions

In [ ]:
rec_days_trials = pd.merge(beh_df, recording_days, left_on = ['mouse_name', 'session_date'],
                           right_on = ['mouse_name', 'date'], how = 'inner')

avg_tb_proportions = []
std_tb_proportions = []

avg_vb_proportions = []
std_vb_proportions = []
for mouse in rec_days_trials['mouse_name'].drop_duplicates():
    mouse_rec_days = rec_days_trials[rec_days_trials['mouse_name'] == mouse]
    t_block = mouse_rec_days['block_type'] == 'Whisker'
    v_block = mouse_rec_days['block_type'] == 'Visual'
    v_trial = mouse_rec_days['trial_type'].str.contains('Vis')
    t_trial = mouse_rec_days['trial_type'].str.contains('Som')
    lick = mouse_rec_days['response'] != 0
    correct = mouse_rec_days['correct'] ==1

    tb_hits = t_block & lick & correct
    tb_misses = t_block & ~lick & ~correct
    tb_FA = t_block & lick & ~correct
    tb_CR = t_block & ~lick & correct

    vb_hits = v_block & lick & correct
    vb_misses = v_block & ~lick & ~correct
    vb_FA = v_block & lick & ~correct
    vb_CR = v_block & ~lick & correct

    
    tb_trial_types = [tb_hits, tb_misses, tb_FA, tb_CR]
    vb_trial_types = [vb_hits, vb_misses, vb_FA, vb_CR]
    avg_tb_proportions.append([sum(tt*1)/sum(t_block*1) for tt in tb_trial_types])
    avg_vb_proportions.append([sum(tt*1)/sum(v_block*1) for tt in vb_trial_types])
    std_tb_proportions.append([sum(tt*1)/sum(t_block*1) for tt in tb_trial_types])
    std_vb_proportions.append([sum(tt*1)/sum(t_block*1) for tt in vb_trial_types])

avg_tb_proportions = np.array(avg_tb_proportions).mean(axis = 0)
avg_vb_proportions = np.array(avg_vb_proportions).mean(axis = 0)
std_tb_proportions = np.array(std_tb_proportions).std(axis = 0)
std_vb_proportions = np.array(std_vb_proportions).std(axis = 0)
trial_type_labels = ['Hits', 'Misses', 'False alarms', 'Correct rejections']

std_tb_proportions

#### calculate performance of mice on each crosmodal training session first for full and then for short stimuli

In [ ]:
cm_training_perf_full = {}
block_training_perf_full = {}
for mouse in beh_df['mouse_name'].unique():
    if first_recording.index.isin([mouse]).any():
        drop_inds = ~((full_stim_performance[mouse].index >= first_CM_days.loc[mouse, 'first_train_date']) & 
                      (full_stim_performance[mouse].index <= first_recording.loc[mouse, 'first_rec_date']))
        cm_training_perf_full[mouse] = full_stim_performance[mouse].drop(np.array(full_stim_performance[mouse].index)[drop_inds])
        block_training_perf_full[mouse] = block_performance_full.loc[mouse].drop(np.array(block_performance_full.loc[mouse].index)[drop_inds])
del cm_training_perf_full['EF0091']
del block_training_perf_full['EF0091'] ## excluded since initially attempted to train on delay task before switching to CM

In [ ]:
cm_training_perf_short = {}
block_training_perf_short = {}

for mouse in beh_df['mouse_name'].unique():
    if short_stim_performance.index.levels[0].isin([mouse]).any():
        drop_inds = ~((short_stim_performance[mouse].index >= first_CM_days.loc[mouse, 'first_train_date']) & 
                      (short_stim_performance[mouse].index <= first_recording.loc[mouse, 'first_rec_date']))
        cm_training_perf_short[mouse] = short_stim_performance[mouse].drop(np.array(short_stim_performance[mouse].index)[drop_inds])
        block_training_perf_short[mouse] = block_performance_short.loc[mouse].drop(np.array(block_performance_short.loc[mouse].index)[drop_inds])
# del cm_training_perf_short['EF0091']
# del block_training_perf_short['EF0091'] ## excluded since initially attempted to train on delay task before switching to CM

#### add days in which no units were recorded and there for dont show up in log_df.

In [ ]:
### days in which no units were recorded dont show up in log_df and therefore must be manually entered
additional_rec_days = {}
additional_rec_days['EF0079'] = ['02-24-16', '02-26-16', '03-23-16', '03-28-16']
additional_rec_days['EF0074'] = []
additional_rec_days['EF0076'] = []
additional_rec_days['EF0081'] = []
additional_rec_days['EF0077'] = ['05-12-16']
additional_rec_days['EF0084'] = ['07-26-16']
additional_rec_days['EF0083'] = ['05-25-16','05-27-16','05-28-16','06-01-16','06-02-16','06-05-16','06-07-16']
additional_rec_days['EF0085'] = ['08-02-16','08-16-16','08-18-16','08-19-16']
additional_rec_days['EF0089'] = ['09-14-16', '09-06-16']
additional_rec_days['EF0088'] = []
additional_rec_days['EF0099'] = []
additional_rec_days['EF0101'] = []
additional_rec_days['EF0102'] = ['08-17-17', '08-21-17']

In [ ]:
for mouse in list(cm_training_perf_full.keys()):
    if mouse in additional_rec_days:
        add_days = pd.concat((pd.DataFrame([mouse]*len(additional_rec_days[mouse]), columns = ['mouse_name']), 
                                           pd.DataFrame(additional_rec_days[mouse], columns = ['date'])), axis = 1)
        add_days['date'] = add_days['date'].apply(lambda x:  datetime.strptime(x, '%m-%d-%y'))
        recording_days = recording_days.append(add_days)
recording_days = recording_days.sort_values(by=['mouse_name', 'date'])

#### EF0101 EF0102 underwent headcap surgery then trained to criterion then inplanted. After implantation the mouse needed a few days to recover performance leading to significantly longer time to criterion. Will remove these recovery days for the the next analysis:

In [ ]:
cm_training_perf_full['EF0101'] = cm_training_perf_full['EF0101'][cm_training_perf_full['EF0101'].index <= '2017-07-21']
cm_training_perf_full['EF0102'] = cm_training_perf_full['EF0102'][cm_training_perf_full['EF0102'].index < '2017-07-27']
block_training_perf_full['EF0101']  = block_training_perf_full[mouse][block_training_perf_full[mouse].index <= '2017-07-21']
block_training_perf_full['EF0102']  = block_training_perf_full[mouse][block_training_perf_full[mouse].index < '2017-07-27']

In [ ]:
recording_days = recording_days.reset_index(drop=True)

In [ ]:
np.array([str(mouse) + ' ' + str(len(cm_training_perf_full[mouse])) for mouse in list(cm_training_perf_full.keys())])

In [ ]:
block_performance_full.columns.rename('index', inplace = True)
block_performance_full = block_performance_full.reset_index()

block_performance_short.columns.rename('index', inplace = True)
block_performance_short = block_performance_short.reset_index()

full_stim_performance = full_stim_performance.reset_index()
short_stim_performance = short_stim_performance.reset_index()

In [ ]:
performance = pd.merge(block_performance_full, full_stim_performance, on = ['mouse_name', 'session_date'])
performance_short = pd.merge(block_performance_short, short_stim_performance, on = ['mouse_name', 'session_date'])

### Remove all recording days where performance dropped below 65% will not be present in all further analyses
#### Although mice reach criterion there would be the occasional day after recording phase began where mice would perform badly. This is possibly due to excessive water restriction, insufficient water restriction, or other difficult to control factors. Since we only care to see how the neurons are responding to the task variables in highly performing mice we chose to exclude these 'off days' from further analysis

In [ ]:
recording_day_inds = pd.merge(performance.reset_index(),recording_days, left_on = ['mouse_name', 'session_date'], 
         right_on = ['mouse_name', 'date'])['index'].as_matrix()

performance['rec_day'] = 0
performance.loc[recording_day_inds, 'rec_day'] = 1
performance.loc[(performance['correct'] < 0.65), 'rec_day'] = 0

rec_performance = performance[performance['rec_day'] == 1]
recording_day_inds_short = pd.merge(performance_short.reset_index(), rec_performance, 
                                    on = ['mouse_name', 'session_date'])['index'].as_matrix()
performance_short['rec_day'] = 0
performance_short.loc[recording_day_inds_short, 'rec_day'] = 1

In [ ]:
mpl.close('all')

# mouse_list.remove('EF0098')
# mouse_list.remove('EF0094')

## to plot all mice:
#mouse_list = list(cm_training_perf_full.keys())

##to plot example mice:
mouse_list = ['EF0074', 'EF0099', 'EF0089']
time_to_crit = np.array([len(cm_training_perf_full[mouse]) for mouse in mouse_list])
ordered_mice = np.array(mouse_list)[np.argsort(time_to_crit)]

fig1a = mpl.figure(figsize = (16,len(mouse_list)*4))
gs1 = gridspec.GridSpec(len(mouse_list),1)
gs2 = gridspec.GridSpec(len(mouse_list),1)
gs3 = gridspec.GridSpec(1,1)
gs4 = gridspec.GridSpec(2,1)


gs1.update(left = 0.2, right = 0.4, hspace = 0.5)
gs2.update(left = 0.42, right = 0.55, hspace = 0.5)
gs3.update(bottom = 0.69, left = 0.65, right = 0.85, hspace = 0.5)
gs4.update(bottom = 0.1, top = 0.6, left = 0.65, right = 0.85, hspace = 0.7)


for i, mouse in enumerate(ordered_mice):
    
    mouse_rec_perf = performance[(performance['rec_day'] == 1) & (performance['mouse_name'] == mouse)] 
    mouse_rec_perf_short = performance_short[(performance_short['rec_day'] == 1) & (performance_short['mouse_name'] == mouse)]

    ax1 = mpl.subplot(gs1[i, 0])
    xmax_tr = len(cm_training_perf_full[mouse])+1
    ax1.plot(range(1,xmax_tr), block_training_perf_full[mouse]['Whisker'], '-oC0', markersize = 3, alpha = 0.8)
    ax1.plot(range(1,xmax_tr), block_training_perf_full[mouse]['Visual'], '-oC1', markersize = 3, alpha = 0.8)
    ax1.plot(range(1,xmax_tr), cm_training_perf_full[mouse], '-ok')

    
    ax2 = mpl.subplot(gs2[i, 0])
    xmax_rec = mouse_rec_perf.shape[0]
    ax2.plot(range(1, xmax_rec+1,1), mouse_rec_perf['correct'], '-om', linewidth = 3)
    ax2.plot(range(1, xmax_rec+1,1), mouse_rec_perf['Whisker'], '-oC0', markersize = 3, alpha = 0.8)
    ax2.plot(range(1, xmax_rec+1,1), mouse_rec_perf['Visual'], '-oC1', markersize = 3, alpha = 0.8)

    
    for ax in [ax1, ax2]:

        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.plot([0, 45], [0.7, 0.7], '--k')
        ax.plot([0, 45], [0.5, 0.5], '--C7', alpha = 0.8)

        ax.set_ylim(0.3,1)
        ax.set_xlim(0,35)
        
        ax.set_ylabel('Fraction correct')
        ax.set_xlabel('Training sessions')
    ax2.set_xlim(0,20)
    ax2.spines['left'].set_visible(False)
    ax2.axes.get_yaxis().set_visible(False)
    ax2.set_xlabel('Test sessions')

    
ax3 = mpl.subplot(gs3[0, 0])
time_to_crit_all = np.array([len(cm_training_perf_full[mouse]) for mouse in list(cm_training_perf_full.keys())])
X2 = np.sort(time_to_crit_all)
F2 = np.array(range(len(time_to_crit_all)))/len(time_to_crit_all)

ax3.plot(X2, F2, 'k', linewidth = 4)
ax3.set_xlabel('Test sessions')
ax3.set_ylabel('Fraction of mice')
ax3.text(.85, .25, 'n =' + str(len(time_to_crit_all)), size=20, weight = 'bold', transform=ax3.transAxes, color='k')
ax3.set_ylim(0,1)
ax3.set_xlim(2,40)

ax4 = mpl.subplot(gs4[0, 0])

full_mouse_list = list(cm_training_perf_full.keys())

for mouse in full_mouse_list:
    mouse_rec_perf = performance[(performance['rec_day'] == 1) & (performance['mouse_name'] == mouse)] 
    mouse_rec_perf_short = performance_short[(performance_short['rec_day'] == 1) & (performance_short['mouse_name'] == mouse)]
    xvals = [0,1]
    rec_block_perf = mouse_rec_perf[['Whisker', 'Visual']]

    if mouse in block_performance_short['mouse_name'].as_matrix():
        mouse_rec_perf_short.columns = ['mouse_name', 'session_date','short_vis', 'short_whisker', 'correct', 'rec_days']
        rec_block_perf = pd.concat((rec_block_perf,  mouse_rec_perf_short[['short_vis', 'short_whisker']]), axis = 1)
        rec_block_perf = rec_block_perf[['Whisker', 'Visual', 'short_whisker', 'short_vis']]
        xvals = [0,1,2,3]
        m_rec_block_perf = rec_block_perf.mean(axis = 0)
        ax4.scatter([3], m_rec_block_perf['short_vis'], color = '#feb24c', s = 70)
        ax4.scatter([2], m_rec_block_perf['short_whisker'], color = 'c', s = 70)
        
    m_rec_block_perf = rec_block_perf.mean(axis = 0)
    ax4.plot(xvals, m_rec_block_perf, 'C7', zorder = 0, alpha = 0.5)
    ax4.scatter([1], m_rec_block_perf['Visual'], color = 'C1', s = 70)
    ax4.scatter([0], m_rec_block_perf['Whisker'], color = 'C0', s = 70)
ax4.plot([-0.5, 3.5], [0.7, 0.7], '--k')
ax4.plot([-0.5, 3.5], [0.5, 0.5], '--C7', alpha = 0.8)
ax4.set_ylim(0.3,1)
ax4.set_xlim(-0.5,3.5)
ax4.set_ylabel('Fraction correct')
mpl.xticks([0,1,2,3], ['Long\ntouch', 'Long\nvisual', 'Short\ntouch', 'Short\nvisual'], rotation=70)

ax5 = mpl.subplot(gs4[1, 0])

colors = ['C2', 'C7', 'C3', 'C4']
bottom = 0
for tt in range(len(avg_tb_proportions)):
    ax5.bar(0, avg_tb_proportions[tt], bottom = bottom, color = colors[tt], yerr = std_tb_proportions[tt])
    ax5.text(.85, .3+.12*tt, trial_type_labels[tt], size=20, weight = 'bold', transform=ax5.transAxes, color = colors[tt])
    bottom = bottom + avg_tb_proportions[tt]
    
bottom = 0
for tt in range(len(avg_vb_proportions)):
    ax5.bar(1, avg_vb_proportions[tt], bottom = bottom, color = colors[tt], yerr = std_vb_proportions[tt])
    bottom = bottom + avg_vb_proportions[tt]
ax5.set_xlim(-1,2)
ax5.set_ylabel('Fraction of trials')
mpl.xticks([0,1], ['Touch\nblock', 'Visual\nblock'])


for ax in [ax3,ax4, ax5]:
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)


fig1a

In [ ]:
fig1a.savefig('fig1a.pdf', format='pdf', dpi=900)

In [ ]:
log_df.columns

In [ ]:
recording_day_RTs = log_df.drop(['cluster_name', 'genotype', 'contingencies', 'spike_times(stim_aligned)',
                                 'spike_times(last_lick_aligned)', 'identified'], axis = 1)
recording_day_RTs.drop_duplicates(['mouse_name', 'date', 'trial_num'], inplace = True)
recording_day_RTs.reset_index(drop = True, inplace = True) 

In [ ]:
recording_day_RTs

In [ ]:
recording_day_RTs['trial_outcome'] = np.nan

long_stim = ~recording_day_RTs['trial_type'].str.contains('1Cyc')
touch_stim = recording_day_RTs['trial_type'].str.contains('Som')
touch_block = recording_day_RTs['block_type'] == 'Whisker'
correct = recording_day_RTs['correct']

recording_day_RTs.loc[(long_stim & touch_stim & touch_block & correct), 'trial_outcome'] = 'touch_hit_long'
recording_day_RTs.loc[(~long_stim & touch_stim & touch_block & correct), 'trial_outcome'] = 'touch_hit_short'
recording_day_RTs.loc[(long_stim & ~touch_stim & ~touch_block & correct), 'trial_outcome'] = 'visual_hit_long'
recording_day_RTs.loc[(~long_stim & ~touch_stim & ~touch_block & correct), 'trial_outcome'] = 'visual_hit_short'

trial_type_RTs = pd.pivot_table(recording_day_RTs, index = ['mouse_name', 'date', 'trial_outcome'],
               values = 'first_lick', aggfunc = np.median).reset_index()

In [ ]:

mpl.close('all')
fig1a_1 = mpl.figure(figsize = (5,4))
ax = fig1a_1 .add_subplot(111)

colors = [ 'C0','C1', 'c', '#feb24c' ]
titles = ['Touch long', 'Visual long', 'Touch short', 'Touch long']

t_long = trial_type_RTs.loc[trial_type_RTs['trial_outcome'] == 'touch_hit_long', 'first_lick']
t_short = trial_type_RTs.loc[trial_type_RTs['trial_outcome'] == 'touch_hit_short', 'first_lick']
v_long = trial_type_RTs.loc[trial_type_RTs['trial_outcome'] == 'visual_hit_long', 'first_lick']
v_short = trial_type_RTs.loc[trial_type_RTs['trial_outcome'] == 'visual_hit_short', 'first_lick']

bplot = ax.boxplot([t_long, v_long, t_short, v_short],labels = titles, vert = False, patch_artist=True)


for patch, color in zip(bplot['boxes'], colors):
        patch.set_facecolor(color)

mpl.setp(bplot['medians'], color='k')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xlim([0,1.2])
ax.set_xlabel('Reaction time (s)')   
fig1a_1 

In [ ]:
fig1a_1.savefig('fig1a_1.pdf', format='pdf', dpi=900)

In [ ]:
recording_days[recording_days['mouse_name'] == 'EF0099']

In [ ]:
recording_days['date'] = recording_days['date'].apply(lambda x: x.date())

In [ ]:
beh_df['date'] = beh_df['session_date'].apply(lambda x: x.date())
test_day = pd.merge(beh_df, recording_days.loc[210].to_frame().T, on = ['mouse_name', 'date'], how = 'inner').sort_values(by= 'trial_num')

t_block = test_day['block_type'] == 'Whisker'
v_block = test_day['block_type'] == 'Visual'

t_trial = test_day['trial_type'].str.contains('Som')
v_trial = test_day['trial_type'].str.contains('Vis')

lick = test_day['response'] != 0
correct = test_day['correct'] == 1

touch_lick = (t_trial & lick)*1
visual_lick =  (v_trial & lick)*1
no_lick = (~lick)*1


cTL = (t_block & t_trial & lick & correct)*1
eTL = (v_block & t_trial & lick & ~correct) | (t_block & t_trial & lick & ~correct)*1

cVL = (v_block & v_trial & lick & correct)*1
eVL = (t_block & v_trial & lick & ~correct) | (v_block & v_trial & lick & ~correct)*1

tNL = (t_trial & ~lick)*1
vNL = (v_trial & ~lick)*1


In [ ]:
# block_edges = np.abs(np.diff(t_block*1))
# block_edges = np.where(np.insert(block_edges, 0, 1))[0]
# block_lick_index = []
# for edge in range(1,len(block_edges)):
#     block_inds = block_edges[edge-1:edge+1]
#     block_t_licks = sum(touch_lick[block_inds[0]:block_inds[1]])
#     block_t_nLicks = sum((t_trial & no_lick)[block_inds[0]:block_inds[1]])
#     block_v_licks = sum(visual_lick[block_inds[0]:block_inds[1]])
#     block_v_nLicks = sum((v_trial & no_lick)[block_inds[0]:block_inds[1]])
#     t_lick_rate = block_t_licks/(block_t_licks+block_t_nLicks)
#     v_lick_rate = block_v_licks/(block_v_licks+block_v_nLicks)
    
#     block_lick_index.append((t_lick_rate-v_lick_rate)/(t_lick_rate+v_lick_rate))
# block_lick_index

In [ ]:
9.363/(8.111/1.663)

In [ ]:
mpl.close('all')

w = 15
tL_height = np.convolve(touch_lick, [1/w]*w, 'valid')
vL_height = np.convolve(visual_lick, [1/w]*w, 'valid')
tNL_height = np.convolve(tNL, [1/w]*w, 'valid')
vNL_height = np.convolve(vNL, [1/w]*w, 'valid')

lick_height = vL_height +tL_height

fig1b = mpl.figure(figsize = (12,3.5))
fig1b.subplots_adjust(bottom = 0.2)
ax = fig1b.add_subplot(111)

g0 = -.5*(vL_height + tL_height)
g0 = [0] * len(lick_height) 
fs = [tNL_height, tL_height, vL_height, vNL_height]

xvals = np.arange(0,len(lick_height))

g_n = g0

labels =  ['Touch-no lick', 'Touch-lick', 'Visual-lick', 'Visual-no lick' ]
colors = [[0.7]*3, 'C0',  'C1', [0.5]*3]
for i,g_i in enumerate(fs):
    
    
    color = colors[i]

    ax.fill_between(xvals, g_n, g_n+g_i, color = colors[i], alpha = 0.5)
    if i == 1:
        ax.plot(xvals, g_n, color = colors[i])
 
    elif i == 3:
        ax.plot(xvals, g_n, color = colors[i-1])

    g_n = g_n + g_i
    ax.text(.85, .1+.12*i, labels[i], size=20, weight = 'bold', transform=ax.transAxes, color = colors[i])

    
xoffset = np.floor((t_block.size - len(xvals))/2)
val_t_block = t_block.as_matrix()[xoffset: xoffset+len(xvals)]
block_changes = np.concatenate([[0],  np.where(np.diff(val_t_block*1))[0] ,[len(val_t_block)]])

block = 0
if t_block.iloc[3] == 1:
    block_colors = ['C0', 'C1']
else:
    block_colors = ['C1', 'C0']
for i in range(len(block_changes)-1):
    ax.add_patch(patches.Rectangle((block_changes[i],1.1),  block_changes[i+1]-block_changes[i],
                                   .1, facecolor = block_colors[i%2], alpha = 0.5))
    block = block + 1

ax.text(.05, .9, 'Touch block', size=20, weight = 'bold', transform=ax.transAxes, color = 'C0', alpha = 0.5)
ax.text(.3, .9, 'Visual block', size=20, weight = 'bold', transform=ax.transAxes, color = 'C1', alpha = 0.5)
    

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_ylim(0,1.4)
ax.spines['bottom'].set_position(('axes', -0.1))
ax.set_yticks([0,0.5,1])
ax.spines['bottom'].set_bounds(150, 200)
ax.spines['left'].set_bounds(0, 1)

ax.set_xlim(-5,300)
ax.set_ylabel('Fraction of trials')

ax.tick_params(axis=u'both', which=u'both',length=0)
mpl.xticks([175],['50 trials'])
ax.yaxis.set_label_coords(-.05, .35)

fig1b

In [ ]:
# import matplotlib.colors
# bl = np.array([31,119,180])/256
# ora = np.array([255,127,14])/256
# cmap = matplotlib.colors.LinearSegmentedColormap('', [[bl, 'k', ora]], N=256, gamma=1.0)

In [ ]:
fig1b.savefig('fig1b.pdf', format='pdf', dpi=900)

In [ ]:
mouse = test_day['mouse_name'].iloc[0]
last_rec_day = recording_days[recording_days['mouse_name'] == mouse].iloc[-1]['date']
ex_mouse = beh_df[beh_df['mouse_name'] == mouse].sort_values(by = ['session_date', 'trial_num'])
ex_mouse = ex_mouse[(ex_mouse['session_date'] >= first_CM_days.loc[mouse][0]) & (ex_mouse['session_date'] <= last_rec_day)]

In [ ]:

t_block = ex_mouse['block_type'] == 'Whisker'
v_block = ex_mouse['block_type'] == 'Visual'

t_trial = ex_mouse['trial_type'].str.contains('Som')
v_trial = ex_mouse['trial_type'].str.contains('Vis')

lick = ex_mouse['response'] != 0
correct = ex_mouse['correct'] == 1

touch_lick = (t_trial & lick)*1
visual_lick =  (v_trial & lick)*1
no_lick = (~lick)*1


cTL = (t_block & t_trial & lick & correct)*1
eTL = ((v_block & t_trial & lick & ~correct) | (t_block & t_trial & lick & ~correct))*1

cVL = (v_block & v_trial & lick & correct)*1
eVL = ((t_block & v_trial & lick & ~correct) | (v_block & v_trial & lick & ~correct))*1

tNL = (t_trial & ~lick)*1
vNL = (v_trial & ~lick)*1


In [ ]:
att_licks = ((t_block & t_trial & lick) | (v_block & v_trial & lick))*1
unatt_licks = ((v_block & t_trial & lick) | (t_block & v_trial & lick))*1

att_noLicks = ((t_block & t_trial & ~lick) | (v_block & v_trial & ~lick))*1
unatt_noLicks = ((v_block & t_trial & ~lick) | (t_block & v_trial & ~lick))*1

pw = 400
run_sum_aL = np.convolve(att_licks , [1]*pw, 'valid')
run_sum_uL = np.convolve(unatt_licks, [1]*pw, 'valid')

run_sum_aNL = np.convolve(att_noLicks , [1]*pw, 'valid')
run_sum_uNL = np.convolve(unatt_noLicks, [1]*pw, 'valid')

block_hit_r = run_sum_aL/(run_sum_aL+run_sum_aNL)
block_FA_r = run_sum_uL/(run_sum_aL+run_sum_uNL)

In [ ]:
mpl.close('all')

w = 50
tL_height = np.convolve(touch_lick, [1/w]*w, 'valid')
vL_height = np.convolve(visual_lick, [1/w]*w, 'valid')
tNL_height = np.convolve(tNL, [1/w]*w, 'valid')
vNL_height = np.convolve(vNL, [1/w]*w, 'valid')

lick_height = vL_height +tL_height

fig1b_2 = mpl.figure(figsize = (24,5))

gs1 = gridspec.GridSpec(1,1)
gs2 = gridspec.GridSpec(1,1)

gs1.update(left = 0.2, right = 0.8, bottom = 0.5, top = 0.9, hspace = 0.5)
gs2.update(left = 0.2, right = 0.8, bottom = 0.2, top = 0.49, hspace = 0.5)

ax = mpl.subplot(gs1[0, 0])
ax1 = mpl.subplot(gs2[0, 0])

g0 = -.5*(vL_height + tL_height)
g0 = [0] * len(lick_height) 
fs = [tNL_height, tL_height, vL_height, vNL_height]

xvals = np.arange(0,len(lick_height))

g_n = g0

labels =  ['Touch-no lick', 'Touch-lick', 'Visual-lick', 'Visual-no lick' ]
colors = [[0.7]*3, 'C0',  'C1', [0.5]*3]
for i,g_i in enumerate(fs):
    
    color = colors[i]

    ax.fill_between(xvals, g_n, g_n+g_i, color = colors[i], alpha = 0.5)

    g_n = g_n + g_i

    
xoffset = np.floor((t_block.size - len(xvals))/2)
val_t_block = t_block.as_matrix()[xoffset: xoffset+len(xvals)]
block_changes = np.concatenate([[0],  np.where(np.diff(val_t_block*1))[0] ,[len(val_t_block)]])

block = 0
if t_block.iloc[3] == 1:
    block_colors = ['C0', 'C1']
else:
    block_colors = ['C1', 'C0']
for i in range(len(block_changes)-1):
    ax.add_patch(patches.Rectangle((block_changes[i],1.1),  block_changes[i+1]-block_changes[i],
                                   .3, facecolor = block_colors[i%2], alpha = 0.5))
    block = block + 1

    
late_session_loc = np.where(ex_mouse['session_date'] == test_day['session_date'][0])[0]
late_session_st = min(late_session_loc)
late_session_end = max(late_session_loc)

early_session_loc = np.where(ex_mouse['session_date'] == ex_mouse['session_date'].unique()[4])[0]
early_session_st = min(early_session_loc)
early_session_end = max(early_session_loc)

mid_session_loc = np.where(ex_mouse['session_date'] == ex_mouse['session_date'].unique()[29])[0]
mid_session_st = min(mid_session_loc)
mid_session_end = max(mid_session_loc)

first_rec = recording_days.loc[recording_days['mouse_name'] == ex_mouse['mouse_name'].iloc[0], 'date'].min()
first_rec_start = np.where(ex_mouse['date'] == first_rec)[0].min()


ax.plot([late_session_st]*2, [1, 1.25], linewidth = 3, color = 'k')
ax.plot([late_session_end]*2, [1, 1.25], linewidth = 3, color = 'k')
ax.plot([early_session_st]*2, [1, 1.25], linewidth = 3, color = 'k')
ax.plot([early_session_end]*2, [1, 1.25], linewidth = 3, color = 'k')
ax.plot([mid_session_st]*2, [1, 1.25], linewidth = 3, color = 'k')
ax.plot([mid_session_end]*2, [1, 1.25], linewidth = 3, color = 'k')
ax.plot([first_rec_start]*2, [0,1.25], linewidth = 3, color ='m')

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.set_ylim(0,1.8)
ax.spines['bottom'].set_position(('axes', -0.1))
ax.axis('off')
ax.set_xlim(-5, len(xvals))

ax1.plot(np.arange(pw/2,len(block_hit_r)+pw/2,1), block_hit_r, 'C2')
ax1.plot(np.arange(pw/2,len(block_FA_r)+pw/2,1), block_FA_r, 'r')
ax1.plot([pw/2,len(block_FA_r)+pw/2], [0.5,0.5], '--k')
ax1.set_xlim(xvals[[0,-1]])
ax1.set_ylim(0,1)
ax1.spines['right'].set_visible(False)
ax1.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
mpl.xticks([8500],['1000 trials'])
ax1.tick_params(axis=u'both', which=u'both',length=0)
ax1.spines['bottom'].set_bounds(8000, 9000)
ax1.spines['left'].set_bounds(0, 1)
ax1.text(.5, .79, 'Hit rate', size=20, weight = 'bold', transform=ax1.transAxes, color = 'C2')
ax1.text(.48, .3, 'False alarm rate', size=20, weight = 'bold', transform=ax1.transAxes, color = 'r')
ax1.plot([first_rec_start]*2, [0,1], linewidth = 3, color ='m')


fig1b_2


In [ ]:
fig1b_2.savefig('fig1b_2.png', format='png', dpi=900)

In [ ]:
ex_mouse[ex_mouse['session_date'] == ex_mouse['session_date'].unique()[29]]

In [ ]:
# recording_days['date'] = recording_days['date'].apply(lambda x: x.date())
beh_df['date'] = beh_df['session_date'].apply(lambda x: x.date())
training_day = ex_mouse[ex_mouse['session_date'] == ex_mouse['session_date'].unique()[15]]

t_block = training_day['block_type'] == 'Whisker'
v_block = training_day['block_type'] == 'visual'

t_trial = training_day['trial_type'].str.contains('Som')
v_trial = training_day['trial_type'].str.contains('Vis')

lick = training_day['response'] != 0
correct = training_day['correct'] == 1

touch_lick = (t_trial & lick)*1
visual_lick =  (v_trial & lick)*1
no_lick = (~lick)*1


cTL = (t_block & t_trial & lick & correct)*1
eTL = (v_block & t_trial & lick & ~correct) | (t_block & t_trial & lick & ~correct)*1

cVL = (v_block & v_trial & lick & correct)*1
eVL = (t_block & v_trial & lick & ~correct) | (v_block & v_trial & lick & ~correct)*1

tNL = (t_trial & ~lick)*1
vNL = (v_trial & ~lick)*1


In [ ]:
mpl.close('all')

w = 15
tL_height = np.convolve(touch_lick, [1/w]*w, 'valid')
vL_height = np.convolve(visual_lick, [1/w]*w, 'valid')
tNL_height = np.convolve(tNL, [1/w]*w, 'valid')
vNL_height = np.convolve(vNL, [1/w]*w, 'valid')

lick_height = vL_height +tL_height

fig1b_3 = mpl.figure(figsize = (12,3.5))
fig1b_3.subplots_adjust(bottom = 0.2)
ax = fig1b_3.add_subplot(111)

g0 = -.5*(vL_height + tL_height)
g0 = [0] * len(lick_height) 
fs = [tNL_height, tL_height, vL_height, vNL_height]

xvals = np.arange(0,len(lick_height))

g_n = g0

labels =  ['Touch-no lick', 'Touch-lick', 'Visual-lick', 'Visual-no lick' ]
colors = [[0.7]*3, 'C0',  'C1', [0.5]*3]
for i,g_i in enumerate(fs):
    
    
    color = colors[i]

    ax.fill_between(xvals, g_n, g_n+g_i, color = colors[i], alpha = 0.5)
    if i == 1:
        ax.plot(xvals, g_n, color = colors[i])
 
    elif i == 3:
        ax.plot(xvals, g_n, color = colors[i-1])

    g_n = g_n + g_i
    ax.text(.85, .1+.12*i, labels[i], size=20, weight = 'bold', transform=ax.transAxes, color = colors[i])

    
xoffset = np.floor((t_block.size - len(xvals))/2)
val_t_block = t_block.as_matrix()[xoffset: xoffset+len(xvals)]
block_changes = np.concatenate([[0],  np.where(np.diff(val_t_block*1))[0] ,[len(val_t_block)]])

block = 0
if t_block.iloc[3] == 1:
    block_colors = ['C0', 'C1']
    block_label = ['Touch block', 'Visual block']
else:
    block_colors = ['C1', 'C0']
for i in range(len(block_changes)-1):
    ax.add_patch(patches.Rectangle((block_changes[i],1.1),  block_changes[i+1]-block_changes[i],
                                   .1, facecolor = block_colors[i%2], alpha = 0.5))
    block = block + i

ax.text(.03, .9, block_label[0], size=20, weight = 'bold', transform=ax.transAxes, color = block_colors[0], alpha = 0.5)
ax.text(.28, .9, block_label[1], size=20, weight = 'bold', transform=ax.transAxes, color = block_colors[1], alpha = 0.5)
    

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_ylim(0,1.4)
ax.spines['bottom'].set_position(('axes', -0.1))
ax.set_yticks([0,0.5,1])
ax.spines['bottom'].set_bounds(100, 150)
ax.spines['left'].set_bounds(0, 1)

ax.set_xlim(-5,270)
ax.set_ylabel('Fraction of trials')

ax.tick_params(axis=u'both', which=u'both',length=0)
mpl.xticks([125],['50 trials'])
ax.yaxis.set_label_coords(-.05, .35)

fig1b_3

In [ ]:
fig1b_3.savefig('fig1b_3.pdf', format='pdf', dpi=900)

In [ ]:
# recording_days['date'] = recording_days['session_date'].apply(lambda x: x.date())
beh_df['date'] = beh_df['session_date'].apply(lambda x: x.date())
training_day = ex_mouse[ex_mouse['session_date'] == ex_mouse['session_date'].unique()[16]]

t_block = training_day['block_type'] == 'Whisker'
v_block = training_day['block_type'] == 'visual'

t_trial = training_day['trial_type'].str.contains('Som')
v_trial = training_day['trial_type'].str.contains('Vis')

lick = training_day['response'] != 0
correct = training_day['correct'] == 1

touch_lick = (t_trial & lick)*1
visual_lick =  (v_trial & lick)*1
no_lick = (~lick)*1


cTL = (t_block & t_trial & lick & correct)*1
eTL = (v_block & t_trial & lick & ~correct) | (t_block & t_trial & lick & ~correct)*1

cVL = (v_block & v_trial & lick & correct)*1
eVL = (t_block & v_trial & lick & ~correct) | (v_block & v_trial & lick & ~correct)*1

tNL = (t_trial & ~lick)*1
vNL = (v_trial & ~lick)*1


In [ ]:
mpl.close('all')

w = 15
tL_height = np.convolve(touch_lick, [1/w]*w, 'valid')
vL_height = np.convolve(visual_lick, [1/w]*w, 'valid')
tNL_height = np.convolve(tNL, [1/w]*w, 'valid')
vNL_height = np.convolve(vNL, [1/w]*w, 'valid')

lick_height = vL_height +tL_height

fig1b_4 = mpl.figure(figsize = (12,3.5))
fig1b_4.subplots_adjust(bottom = 0.2)
ax = fig1b_4.add_subplot(111)

g0 = -.5*(vL_height + tL_height)
g0 = [0] * len(lick_height) 
fs = [tNL_height, tL_height, vL_height, vNL_height]

xvals = np.arange(0,len(lick_height))

g_n = g0

labels =  ['Touch-no lick', 'Touch-lick', 'Visual-lick', 'Visual-no lick' ]
colors = [[0.7]*3, 'C0',  'C1', [0.5]*3]
for i,g_i in enumerate(fs):
    
    
    color = colors[i]

    ax.fill_between(xvals, g_n, g_n+g_i, color = colors[i], alpha = 0.5)
    if i == 1:
        ax.plot(xvals, g_n, color = colors[i])
 
    elif i == 3:
        ax.plot(xvals, g_n, color = colors[i-1])

    g_n = g_n + g_i
    ax.text(.85, .1+.12*i, labels[i], size=20, weight = 'bold', transform=ax.transAxes, color = colors[i])

    
xoffset = np.floor((t_block.size - len(xvals))/2)
val_t_block = t_block.as_matrix()[xoffset: xoffset+len(xvals)]
block_changes = np.concatenate([[0],  np.where(np.diff(val_t_block*1))[0] ,[len(val_t_block)]])

block = 0
if t_block.iloc[3] == 1:
    block_colors = ['C0', 'C1']
    block_label = ['Touch block', 'Visual block']
else:
    block_colors = ['C1', 'C0']
for i in range(len(block_changes)-1):
    ax.add_patch(patches.Rectangle((block_changes[i],1.1),  block_changes[i+1]-block_changes[i],
                                   .1, facecolor = block_colors[i%2], alpha = 0.5))
    block = block + 1

ax.text(.03, .9, block_label[0], size=20, weight = 'bold', transform=ax.transAxes, color = block_colors[0], alpha = 0.5)
ax.text(.27, .9, block_label[1], size=20, weight = 'bold', transform=ax.transAxes, color = block_colors[1], alpha = 0.5)
    

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_ylim(0,1.4)
ax.spines['bottom'].set_position(('axes', -0.1))
ax.set_yticks([0,0.5,1])
ax.spines['bottom'].set_bounds(100, 150)
ax.spines['left'].set_bounds(0, 1)

ax.set_xlim(-5,270)
ax.set_ylabel('Fraction of trials')

ax.tick_params(axis=u'both', which=u'both',length=0)
mpl.xticks([125],['50 trials'])
ax.yaxis.set_label_coords(-.05, .35)

fig1b_4

In [ ]:
fig1b_4.savefig('fig1b_4.pdf', format='pdf', dpi=900)